In [1]:
import allel
import h5py
import ingenos
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
%matplotlib inline

##### read in the OpenArray calls

In [2]:
base_dir = "../../../data/amplicon_sequencing/funestus/"

In [3]:
results = pd.read_csv(base_dir + "final_afun_wide.csv", 
                      na_values = -9)
results.set_index("SampleID", inplace=True)
results.head()

,3Ra:10488834_C_T,3Ra:10812270_T_A,3Ra:10870174_C_T,3Ra:11009607_C_G,3Ra:2008898_C_T,3Ra:2055956_T_G,3Ra:2097318_C_G,3Ra:2101786_A_T,3Ra:2486161_A_C,3Ra:3507910_C_G,...,3Rb:21916268_C_T,3Rb:22631746_A_G,3Rb:23021477_T_C,3Rb:23302727_A_G,3Rb:26776397_G_A,3Rb:28574306_C_T,3Rb:28578288_T_C,3Rb:29236594_A_T,3Rb:30063069_C_G,3Rb:30661948_T_C
SampleID,,,,,,,,,,,,,,,,,,,,,
00-00025,1,1.0,1,1,1,1.0,1,1,1.0,1,...,1.0,2.0,2.0,1.0,2.0,2,2,2,2.0,2
00-00032,0,2.0,2,2,2,2.0,2,2,2.0,2,...,1.0,1.0,1.0,1.0,0.0,0,1,0,1.0,1
00-00095,1,1.0,1,1,1,1.0,1,1,1.0,1,...,0.0,1.0,1.0,1.0,0.0,1,1,1,1.0,1
00-00134,0,1.0,1,2,1,1.0,2,2,2.0,1,...,1.0,1.0,1.0,2.0,1.0,1,1,1,NaN,1
00-00136,2,2.0,2,2,1,1.0,2,2,2.0,2,...,0.0,1.0,1.0,2.0,1.0,1,0,0,1.0,2


In [4]:
results.shape

(238, 28)

In [5]:
results = results.T

results["label"] = results.reset_index()["index"].str.split(
    ":", expand=True)[0].str.slice(start = -1).values

results["pos"] = results.reset_index()["index"].str.split(
    ":", expand=True)[1].str.split("_", expand=True)[0].map(int).values

results.set_index(["label","pos"], inplace=True)

results = results.T

results.head()

label           a                                                        \
pos      10488834 10812270 10870174 11009607 2008898  2055956  2097318    
SampleID                                                                  
00-00025      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
00-00032      0.0      2.0      2.0      2.0      2.0      2.0      2.0   
00-00095      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
00-00134      0.0      1.0      1.0      2.0      1.0      1.0      2.0   
00-00136      2.0      2.0      2.0      2.0      1.0      1.0      2.0   

label                                ...        b                             \
pos      2101786  2486161  3507910   ... 21916268 22631746 23021477 23302727   
SampleID                             ...                                       
00-00025      1.0      1.0      1.0  ...      1.0      2.0      2.0      1.0   
00-00032      2.0      2.0      2.0  ...      1.0      1.0      1.0      1.0   
00-00095      1.0      1.0      1.0  ...      0.0      1.0      1.0      1.0   
00-00134      2.0      2.0      1.0  ...      1.0      1.0      1.0      2.0   
00-00136      2.0      2.0      2.0  ...      0.0      1.0      1.0      2.0   

label                                                           
pos      26776397 28574306 28578288 29236594 30063069 30661948  
SampleID                                                        
00-00025      2.0      2.0      2.0      2.0      2.0      2.0  
00-00032      0.0      0.0      1.0      0.0      1.0      1.0  
00-00095      0.0      1.0      1.0      1.0      1.0      1.0  
00-00134      1.0      1.0      1.0      1.0      NaN      1.0  
00-00136      1.0      1.0      0.0      0.0      1.0      2.0  

[5 rows x 28 columns]

##### check missing data by SNP

In [6]:
inversions = results.columns.levels[0]

missing_df = pd.DataFrame(columns = ["label", "n_called"])

for inversion in inversions:
    
    chunk = results.loc[:,inversion]

    n_specs = chunk.shape[0]
    n_snps = chunk.shape[1]
    
    missing_df = missing_df.append(pd.DataFrame({"label" : [inversion] * n_snps,
                                                 "pos" : chunk.columns.values,
                 "n_called" : ((1-(np.sum(chunk.isnull(), axis=0) / n_specs)) * 100).values}))

missing_df.head()

/afs/crc.nd.edu/user/r/rlove1/.local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,label,n_called,pos
0,a,100.000000,10488834.0
1,a,99.159664,10812270.0
2,a,100.000000,10870174.0
3,a,100.000000,11009607.0
4,a,100.000000,2008898.0


In [7]:
missing_df.reset_index(inplace=True, drop=True)
missing_df.drop_duplicates(subset="label").index

Int64Index([0, 16], dtype='int64')

In [8]:
len(missing_df)

28

In [9]:
missing_df.loc[missing_df["n_called"] < 80]

,label,n_called,pos
10,a,74.789916,3847845.0


##### drop this specimen

In [10]:
results.drop(("a", 3847845), axis=1, inplace=True)
results.shape

(238, 27)

In [11]:
results.head()

label           a                                                        \
pos      10488834 10812270 10870174 11009607 2008898  2055956  2097318    
SampleID                                                                  
00-00025      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
00-00032      0.0      2.0      2.0      2.0      2.0      2.0      2.0   
00-00095      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
00-00134      0.0      1.0      1.0      2.0      1.0      1.0      2.0   
00-00136      2.0      2.0      2.0      2.0      1.0      1.0      2.0   

label                                ...        b                             \
pos      2101786  2486161  3507910   ... 21916268 22631746 23021477 23302727   
SampleID                             ...                                       
00-00025      1.0      1.0      1.0  ...      1.0      2.0      2.0      1.0   
00-00032      2.0      2.0      2.0  ...      1.0      1.0      1.0      1.0   
00-00095      1.0      1.0      1.0  ...      0.0      1.0      1.0      1.0   
00-00134      2.0      2.0      1.0  ...      1.0      1.0      1.0      2.0   
00-00136      2.0      2.0      2.0  ...      0.0      1.0      1.0      2.0   

label                                                           
pos      26776397 28574306 28578288 29236594 30063069 30661948  
SampleID                                                        
00-00025      2.0      2.0      2.0      2.0      2.0      2.0  
00-00032      0.0      0.0      1.0      0.0      1.0      1.0  
00-00095      0.0      1.0      1.0      1.0      1.0      1.0  
00-00134      1.0      1.0      1.0      1.0      NaN      1.0  
00-00136      1.0      1.0      0.0      0.0      1.0      2.0  

[5 rows x 27 columns]

In [12]:
results.columns = results.columns.remove_unused_levels()

##### now check specimen call rate

In [13]:
sample_missingness = pd.DataFrame(results.isnull().mean(axis=1))
sample_missingness.columns = ["null_rate"]
sample_missingness.head()

,null_rate
SampleID,
00-00025,0.000000
00-00032,0.000000
00-00095,0.000000
00-00134,0.074074
00-00136,0.000000


In [14]:
1 - sample_missingness.max()

null_rate    0.814815
dtype: float64

In [15]:
1 - sample_missingness.min()

null_rate    1.0
dtype: float64

In [16]:
np.sum(sample_missingness["null_rate"] > 0.2)

0

##### good.

##### calculate genotypes.

In [17]:
columns = [name + "_mean" for name in results.columns.levels[0]]

columns.extend([name + "_called" for name in results.columns.levels[0]])

averaged = pd.DataFrame(index = results.index, columns = columns)

inversions = results.columns.levels[0]

In [18]:
for inversion in inversions:
    
    chunk = results[inversion]
    
    for index, row in chunk.iterrows():
        
        averaged.loc[index, inversion + "_mean"] = np.mean(row)
        averaged.loc[index, inversion + "_called"] =\
        1 - np.mean(row.isnull())
        
averaged.head()

,a_mean,b_mean,a_called,b_called
SampleID,,,,
00-00025,1,1.58333,1,1
00-00032,1.8,0.666667,1,1
00-00095,1,0.666667,1,1
00-00134,1.14286,1,0.933333,0.916667
00-00136,1.86667,0.75,1,1


##### classify the computational score into inversion genotypes

In [19]:
for column in inversions:
    
    new_name = re.sub("_mean", "", column) + "_assigned"
    #print(column, new_name)
    
    karyos = []
    
    for value in averaged[column + "_mean"].values:
        
        if value <= (2/3):
            
            karyos.append(0)
            
        elif value > (2/3) and value <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    averaged[new_name] = karyos
    
averaged.head()

,a_mean,b_mean,a_called,b_called,a_assigned,b_assigned
SampleID,,,,,,
00-00025,1,1.58333,1,1,1,2
00-00032,1.8,0.666667,1,1,2,0
00-00095,1,0.666667,1,1,1,0
00-00134,1.14286,1,0.933333,0.916667,1,1
00-00136,1.86667,0.75,1,1,2,1


##### merge with cytogenetic karyotypes

In [20]:
cyto = pd.read_excel(base_dir + "Database_Afunestus_CC3.xlsx")

cyto.head()

,SpecimenID,Date of collection,Year,Season,Month,Village,Method,Sample,Time-slot,Bait_name,...,3Ra_karyo,3Rb_karyo,X_5A.1,2RAS.1,Inversions Scoring,Karyotype,Rule 1,Rule 2,Rule 3,Rule 4
0,05-00022,2005-02-01,2005,04-05,2,Koubri,OBET,Anthropophilic,NaN,Human,...,2,1,0,Ok,Complete,00210,Folonzo,Folonzo,Folonzo,Folonzo
1,05-00024,2005-02-01,2005,04-05,2,Koubri,OBET,Anthropophilic,NaN,Human,...,1,1,0,Ok,Partial,––110,Rule 2,Rule 3,Folonzo,Folonzo
2,05-00025,2005-02-02,2005,04-05,2,Koubri,OBET,Anthropophilic,NaN,Human,...,2,-9,0,Ok,Partial,102–0,Folonzo,Folonzo,Folonzo,Folonzo
3,05-00027,2005-02-02,2005,04-05,2,Koubri,OBET,Anthropophilic,NaN,Human,...,2,0,0,Ok,Complete,10200,Folonzo,Folonzo,Folonzo,Folonzo
4,05-00028,2005-02-07,2005,04-05,2,Koubri,OBET,Anthropophilic,NaN,Human,...,2,2,0,Ok,Complete,10220,Folonzo,Folonzo,Folonzo,Folonzo


In [21]:
np.sum(cyto["SpecimenID"].isin(averaged.reset_index()["SampleID"]))

238

In [22]:
cyto = cyto.loc[(cyto["SpecimenID"].isin(averaged.reset_index()["SampleID"])),
        ["SpecimenID", "3Ra_karyo", "3Rb_karyo"]]

cyto.rename(columns = {"SpecimenID" : "SampleID",
                      "3Ra_karyo" : "a_cyto",
                      "3Rb_karyo" : "b_cyto"}, inplace=True)

cyto["a_cyto"] = cyto["a_cyto"].map(str)
cyto["b_cyto"] = cyto["b_cyto"].map(str)

cyto.replace("-9", np.nan, inplace=True)

compare = pd.merge(averaged.reset_index(), cyto, on="SampleID")

compare.head()

,SampleID,a_mean,b_mean,a_called,b_called,a_assigned,b_assigned,a_cyto,b_cyto
0,00-00025,1,1.58333,1,1,1,2,1,2
1,00-00032,1.8,0.666667,1,1,2,0,2,1
2,00-00095,1,0.666667,1,1,1,0,1,1
3,00-00134,1.14286,1,0.933333,0.916667,1,1,2,0
4,00-00136,1.86667,0.75,1,1,2,1,2,1
